In [ ]:
# This allows importing Jupyter notebooks as modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import JupyterNotebookImporter

In [ ]:
#import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
import keras.backend as kb
import keras
from sklearn.metrics import mean_squared_error
import numpy as np
from keras import regularizers, optimizers
from numpy.random import seed
import AlgoPlotting as plt
#from AlgoPlotting import XYChart

In [ ]:
def shape_it(X):
    return np.expand_dims(X.reshape((-1,1)),2)

In [ ]:
data = np.random.randint(-5,5,200)

n_data = len(data)
data = np.matrix(data)
n_train = int(0.8*n_data)

index_delay = 5
y_train = shape_it(data[:,:n_train])
x_train = shape_it(data[:,index_delay:(n_train+index_delay)])
y_test = shape_it(data[:,n_train:-index_delay])
x_test = shape_it(data[:,(n_train+index_delay):])

In [ ]:
training_chart = plt.XYChart(y=[x_train, y_train],
                             names=['X Train','Y Train'],
                             title='Training Data'
                            )

In [ ]:
testing_chart = plt.XYChart(y=[x_test, y_test],
                            names=['X Test','Y Test'],
                            title='Testing Data'
                           )

In [ ]:
class SGDLearningRateTracker(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        opt = self.model.optimizer
        
        # Not sure why but eval'ing 1 by 1 then calculating doesn't crash, but 1 line does
        lr = kb.eval(opt.lr)
        decay = kb.eval(opt.decay)
        iterations = kb.eval(opt.iterations)
        #lr = kb.eval(opt.lr * (1. / (1. + opt.decay * opt.iterations))) # This crashes
        lr = (lr * (1. / (1. + decay * iterations)))
        #print('\nLR: {:.6f}\n'.format(lr))
        logs['lr_calc'] = lr

In [ ]:
def train_and_monitor(model, epochs, x_train, y_train, x_test, y_test, epoch_update_interval=10, losses_trim=0):
    predictions = []
    losses = []
    learning_rates = []
    
    predictions_chart = plt.XYChart(title='Predictions', x_label='Index', y_label='Y', names=['Prediction', 'Test Data'])
    losses_chart = plt.XYChart(title='Losses', x_label='Epochs', y_label='Loss')
    dlosses_chart = plt.XYChart(title='Diff(Losses)', x_label='Epochs', y_label='Diff(Loss)')
    lr_chart = plt.XYChart(title='Learning Rate', x_label='Epochs', y_label='Learning Rate')
    
    for epoch in range(epochs):
        history = model.fit(x_train, np.reshape(y_train,(-1,)), epochs=1, batch_size=batch_size, verbose=0, shuffle=False,
                            callbacks=[SGDLearningRateTracker()])
        loss = history.history['loss'][0]
        lr = history.history['lr_calc'][0]
        losses.append(loss)
        learning_rates.append(lr)

        #print('Epoch: ', epoch, ', Loss: ', loss)
        model.reset_states()

        # Update the plots
        if not epoch % epoch_update_interval:
            predictions = []
            for ii in range(len(x_test)):
                # make one-step forecast
                X = x_test[ii]
                X = X.reshape(1, 1, 1)
                y_pred = model.predict(X, batch_size=batch_size)[0,0]

                # store forecast
                predictions.append(y_pred)
                expected = y_test[ii]

            # Push off the starting indices once the losses is long enough
            if len(losses) <= losses_trim:
                y_losses = losses
                x_losses = np.arange(len(y_losses))
            elif len(losses) > losses_trim and len(losses) < 2*losses_trim:
                start = len(losses) - losses_trim
                y_losses = losses[start:]
                x_losses = np.arange(start, len(losses))
            else:
                y_losses = losses[losses_trim:]
                x_losses = np.arange(losses_trim, len(losses))
            
            predictions_chart.update(y=[predictions, y_test])
            losses_chart.update(x=x_losses, y=y_losses)
            dlosses_chart.update(x=x_losses[:-1], y=np.diff(y_losses))
            lr_chart.update(y=learning_rates)
            model.reset_states()

In [ ]:
model = Sequential()
batch_size = 1
los = 'mse'
act = 'relu'
learning_rate = 0.00005
decay_rate = 0.00015 #learning_rate / epochs
momentum = 0.0
sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
opt = sgd

model.add(SimpleRNN(12, batch_input_shape=(batch_size, x_train.shape[1], x_train.shape[2]),stateful=True, activation=act))
model.add(Dense(12, activation=act))
model.add(Dense(1))

#model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.compile(loss=los, optimizer=opt, metrics=['accuracy'])

epochs = 300
epoch_update_interval = 5
losses_trim = 20

train_and_monitor(model, epochs, x_train, y_train, x_test, y_test,
                  epoch_update_interval=epoch_update_interval, losses_trim=losses_trim)

In [ ]:
modelR = Sequential()
batch_size = 1
los = 'mse'
act = 'tanh'
learning_rate = 0.00005
decay_rate = 0.0001
momentum = 0.0
sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
opt = sgd
kreg = regularizers.l2(0.01)
areg = regularizers.l1(0.01)
neurons = 30

use_regularization = True

if use_regularization:
    modelR.add(SimpleRNN(neurons, batch_input_shape=(batch_size, x_train.shape[1], x_train.shape[2]), stateful=True,
                         kernel_regularizer=kreg, activity_regularizer=areg))
    modelR.add(Dense(neurons, kernel_regularizer=kreg, activity_regularizer=areg, activation=act))
    modelR.add(Dense(1,kernel_regularizer=kreg, activity_regularizer=areg))
    modelR.compile(loss=los, optimizer=opt, metrics=['accuracy'])
else:
    modelR.add(SimpleRNN(neurons, batch_input_shape=(batch_size, x_train.shape[1], x_train.shape[2]), stateful=True))
    modelR.add(Dense(neurons, activation=act))
    modelR.add(Dense(1))
    modelR.compile(loss=los, optimizer=opt, metrics=['accuracy'])

epochs = 500
epoch_update_interval = 5
losses_trim = 100

train_and_monitor(modelR, epochs, x_train, y_train, x_test, y_test,
                  epoch_update_interval=epoch_update_interval, losses_trim=losses_trim)

In [ ]:
modelR = Sequential()
batch_size = 1
los = 'mse'
act = 'tanh'
learning_rate = 0.00005
decay_rate = 0.0001
momentum = 0.0
sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
opt = sgd
kreg = regularizers.l2(0.01)
areg = regularizers.l1(0.01)
neurons = 10

use_regularization = False

if use_regularization:
    modelR.add(LSTM(neurons, batch_input_shape=(batch_size, x_train.shape[1], x_train.shape[2]), stateful=True,
                         kernel_regularizer=kreg, activity_regularizer=areg))
    modelR.add(Dense(neurons, kernel_regularizer=kreg, activity_regularizer=areg, activation=act))
    modelR.add(Dense(1,kernel_regularizer=kreg, activity_regularizer=areg))
    modelR.compile(loss=los, optimizer=opt, metrics=['accuracy'])
else:
    modelR.add(SimpleRNN(neurons, batch_input_shape=(batch_size, x_train.shape[1], x_train.shape[2]), stateful=True))
    modelR.add(Dense(neurons, activation=act))
    modelR.add(Dense(1))
    modelR.compile(loss=los, optimizer=opt, metrics=['accuracy'])

epochs = 2000
epoch_update_interval = 5
losses_trim = 100

train_and_monitor(modelR, epochs, x_train, y_train, x_test, y_test,
                  epoch_update_interval=epoch_update_interval, losses_trim=losses_trim)

In [ ]:
# One step forecast on testing data
model.reset_states()
model.predict(X_train, batch_size=batch_size)
predictions = []
for i in range(len(X_test)):
    # make one-step forecast
    X = X_test[i]
    X = X.reshape(1, 1, 1)
    yhat = model.predict(X, batch_size=batch_size)[0,0]

    # store forecast
    predictions.append(yhat)
    expected = Y_test[ i ]
    print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

# report performance
rmse = np.sqrt(mean_squared_error(Y_test.reshape(len(Y_test)), predictions))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# Dynamic forecast on test data
model.reset_states()
model.predict(X_train, batch_size=batch_size)

dynpredictions = list()
dyhat = X_test[0]


for i in range(len(X_test)):
    # make one-step forecast
    dyhat = yhat.reshape(1, 1, 1)
    dyhat = model.predict(dyhat, batch_size=batch_size)[0,0]

    # store forecast
    dynpredictions.append(dyhat)
    expected = Y_test[ i ]
    print('Month=%d, Predicted Dynamically=%f, Expected=%f' % (i+1, dyhat, expected))


drmse = np.sqrt(mean_squared_error(Y_test.reshape(len(Y_test)), dynpredictions))
print('Test Dynamic RMSE: %.3f' % drmse)
# line plot of observed vs predicted
plt.plot(Y_test.reshape(len(Y_test)))
plt.plot(dynpredictions)
plt.show()